# Kubeflow Fairing 라이브러리를 활용한 분산학습 및 서빙 시나리오

Version Details of the ai-devops used in scenario
- kserve v0.10.0
- knative-serving v1.2.5
- training-operator v1.5.0
- notebook-controller b0.2.8
- profile-controller v1.6.1
- cluster-local-gateway v1.14.1

Scenario Process
- Tensorflow 예제를 가져와 분산 학습 지원을 위한 수정사항을 반영한다.
- Kubeflow Fairing을 활용하여 도커 이미지를 빌드하고 모델 학습을 위한 TFJob을 생성한다.
- 학습된 모델을 서빙하기 위한 inferenceservice 역시 fairing을 통하여 생성한다.

### Training Code 작성

본 시나리오에서는 아래 링크의 Tensoflow 예제를 분산 학습과 모델 서빙에 적합한 형태로 수정하였으며 업데이트된 코드는 다음과 같다. [mnist.py](mnist.py). 

**참고** [examples](https://github.com/tensorflow/tensorflow/blob/9a24e8acfcd8c9046e1abaac9dbf5e146186f4c2/tensorflow/examples/learn/mnist.py)

### 필요 라이브러리 확인

ㅇ

In [ ]:
!pip show kubeflow-fairing

### Kubeflow fairing을 위한 Registry와 리소스들이 생성될 네임스페이스를 설정한다.

* 노트북에서 도커 이미지를 빌드하기위하여 이미지가 저장될 레지스트리를 설정한다.
* 단계를 진행하면서 리소스들이 생성될 네임스페이스를 지정한다.

In [2]:
# 이미지 저장을 위한 도커 레지스트리 설정
# 이미지 push를 위한 권한을 보유하고 있는지 확인한다.
# 아래의 변수들을 상황에 맞게 설정해준다.
DOCKER_REGISTRY = ex) '172.21.5.5:5000'

# 네임스페이스 설정
my_namespace = 'demo'

## 모델 저장을 위한 PV/PVC를 생성

Persistent Volume(PV) 와 Persistent Volume Claim(PVC) 를 생성한다.
PVC는 아래 단계에서 학습과 서빙 pod에 의해 사용된다.
**참고** pv/pvc를 생성하기 위해 notebook pod에 할당된 default-editor SA에 kubeflow-admin clusterrole을 바인딩한다.
fairing-demo 디렉토리의 rolebinding-sample.yaml을 활용하여 생성하거나 아래 명령어를 통해 생성한다.
```
kubectl create rolebinding sample-rolebinding --clusterrole=kubeflow-admin --serviceaccount=demo:default-editor --namespace=demo
```

In [3]:
# 분산 트레이닝을 위해서 PVC는 클러스터의 모든 노드에서 접근가능하여야한다.
# 예시에서는 NFS PV를 활용하엿음.
# 아래의 변수들을 상황에 맞게 설정해준다.
## storageclass를 통한 Dynamic Provisioning 환경에서는 fairing-demo 디렉토리 내의 pvc-sample을 활용하여 pvc 생성후 추후 단계에서 생성한 pvc를 이용하여 진행한다.
nfs_server = ex)'172.21.5.5'
nfs_path = ex)'/nfs/test'
pv_name = 'kubeflow-mnist'
pvc_name = 'mnist-pvc'

(Optional) 이미 PV 및 PVC가 생성되어있다면 아래 단계는 건너뛴다. 추후 단계에서 pvc_name이 변수로 사용되는 경우가 있으므로 미리 생성해놓은 PVC의 이름은 위 변수 설정 단계에서 설정한다.

- storageclass를 사용하는 경우에는 pv를 생성하지 않아도 되므로 아래코드를 활용하지 않고 fairing-demo 디렉토리 내의 pvc-sample을 활용하여 생성한다.

- 해당 sample에는 storageclass 필드를 명시하지 않아 자동으로 default StorageClass를 활용하도록 되어있으며 수동지정을 위해서는 spec.storageClassName필드에 원하는 
스토리지클래스를 지정한다.

In [ ]:
from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubeflow.fairing.utils import is_running_in_k8s
import yaml

pv_yaml = f'''
apiVersion: v1
kind: PersistentVolume
metadata:
  name: {pv_name}
spec:
  capacity:
    storage: 10Gi
  accessModes:
  - ReadWriteMany
  persistentVolumeReclaimPolicy: Retain
  nfs:
    path: {nfs_path}
    server: {nfs_server}
'''
pvc_yaml = f'''
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: {pvc_name}
  namespace: {my_namespace}
spec:
  accessModes:
    - ReadWriteMany
  storageClassName: ""
  resources:
    requests:
      storage: 10Gi
'''

if is_running_in_k8s():
    k8s_config.load_incluster_config()
else:
    k8s_config.load_kube_config()

k8s_core_api = k8s_client.CoreV1Api()
k8s_core_api.create_persistent_volume(yaml.safe_load(pv_yaml))
k8s_core_api.create_namespaced_persistent_volume_claim(my_namespace, yaml.safe_load(pvc_yaml))

## Kubeflow Fairing을 사용한 도커 이미지 빌드 및 TFJob을 통한 학습

* kubeflow fairing을 통해 필요한 dependency가 포함된 도커 이미지를 빌드한다.
* 모델 학습을 위해 클러스터에 TFJob을 생성한다.

먼저, 몇가지 parameter들을 설정한다.

In [5]:
num_chief = 1 #number of Chief in TFJob 
num_ps = 1  #number of PS in TFJob 
num_workers = 2  #number of Worker in TFJob 
model_dir = "/mnt"
export_path = "/mnt/export" 
train_steps = "1000"
batch_size = "100"
learning_rate = "0.01"

도커 이미지를 빌드하고 레지스트리에 푸쉬한 뒤 분산 학습을 위한 TFJob을 생성한다.

*kubeflow.fairing.builders.cluster.minio_context가 없다는 에러가 뜬다면, 아래의 코드를 통해 kubeflow-fairing 라이브러리를 업데이트한다.
*에러가 발생하지 않으면 아래 코드는 실행하지 않는다.

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install kubeflow-fairing --upgrade

In [ ]:
import uuid
from kubeflow import fairing   
from kubeflow.fairing.kubernetes.utils import mounting_pvc
from kubeflow.fairing.kubernetes import utils as k8s_utils
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource

minio_context_source = MinioContextSource(endpoint_url='http://minio-service.kubeflow.svc.cluster.local:9000', 
                                                  minio_secret='minio', 
                                                  minio_secret_key='minio123', 
                                                  region_name='us-east-1')

tfjob_name = f'mnist-training-{uuid.uuid4().hex[:4]}'

output_map =  {
    "Dockerfile": "Dockerfile",
    "mnist.py": "mnist.py"
}

command=["python",
         "/opt/mnist.py",
         "--tf-model-dir=" + model_dir,
         "--tf-export-dir=" + export_path,
         "--tf-train-steps=" + train_steps,
         "--tf-batch-size=" + batch_size,
         "--tf-learning-rate=" + learning_rate]

fairing.config.set_preprocessor('python', command=command, path_prefix="/app", output_map=output_map)
fairing.config.set_builder(name='cluster', registry=DOCKER_REGISTRY, base_image="",
                           image_name="mnist", dockerfile_path="Dockerfile", context_source=minio_context_source, push=True)
fairing.config.set_deployer(name='tfjob', namespace=my_namespace, stream_log=False, job_name=tfjob_name,
                            chief_count=num_chief, worker_count=num_workers, ps_count=num_ps, 
                            pod_spec_mutators=[mounting_pvc(pvc_name=pvc_name, pvc_mount_path=model_dir)])
fairing.config.run()

### 생성된 TFJob 조회

In [ ]:
from kubeflow.tfjob import TFJobClient
tfjob_client = TFJobClient()

tfjob_client.get(tfjob_name, namespace=my_namespace)

### Training job의 종료 대기

In [ ]:
tfjob_client.wait_for_job(tfjob_name, namespace=my_namespace, watch=True)

### TFJob 성공여부 확인

In [ ]:
tfjob_client.is_job_succeeded(tfjob_name, namespace=my_namespace)

### Training 로그 조회

In [ ]:
tfjob_client.get_logs(tfjob_name, namespace=my_namespace)

## Kserve를 통한 추론 서비스 배포

In [ ]:
!pip install kserve==0.7.0

In [ ]:
isvc_name = f'mnist-service-{uuid.uuid4().hex[:4]}'
default_storage_uri='pvc://' + pvc_name + '/export'

In [ ]:
from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec


kserve_version='v1beta1'
api_version = constants.KSERVE_GROUP + '/' + kserve_version

isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(
                                   name=isvc_name, namespace=my_namespace),
                               spec=V1beta1InferenceServiceSpec(
                               predictor=V1beta1PredictorSpec(
                               sklearn=(V1beta1SKLearnSpec(
                                   storage_uri=default_storage_uri))))
)




In [ ]:
KServe = KServeClient()
KServe.create(isvc)

### 생성한 inferenceservice 조회

In [ ]:
KServe.get(isvc_name, namespace=my_namespace, watch=True, timeout_seconds=120)

### 추론 서비스 엔드포인트 확인

In [ ]:
mnist_isvc = KServe.get(isvc_name, namespace=my_namespace)
mnist_isvc_name = mnist_isvc['metadata']['name']
mnist_isvc_endpoint = mnist_isvc['status'].get('url', '')
print("MNIST Service Endpoint: " + mnist_isvc_endpoint)

### 추론 서비스에 대한 prediction 요청 실행

In [ ]:
ISTIO_CLUSTER_IP=!kubectl -n istio-system get service ingressgateway -o jsonpath='{.spec.clusterIP}'
CLUSTER_IP=ISTIO_CLUSTER_IP[0]
MODEL_HOST=f"Host: {mnist_isvc_name}.{my_namespace}.example.com"
!curl -v -H "{MODEL_HOST}" http://{CLUSTER_IP}/v1/models/{mnist_isvc_name}:predict -d @./input.json

## 리소스 삭제

TFJob 삭제

In [ ]:
tfjob_client.delete(tfjob_name, namespace=my_namespace)

InferenceService 삭제

In [ ]:
KServe.delete(isvc_name, namespace=my_namespace)
